In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/CAIS

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/CAIS


In [ ]:
import pickle
import numpy as np
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os
import csv

EMBEDDING_DIM = 50
EMBEDDINGS_DIR = './glove.6B.50d.txt'
CSV_DIR = './data.csv'

tweets = []
labels = []
first = True

import csv
with open(CSV_DIR, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      if not first:
        labels.append(row[0])
        tweets.append(row[2])
      else:
        first = False




In [ ]:
labels = np.array(labels)
tweets = np.array(tweets)

Taken from RNN lesson


In [ ]:
#TODO: Shuffle data 
from sklearn.utils import shuffle

training_data_init, training_labels_init = shuffle(tweets, labels, random_state = 0)

In [ ]:
#only use a certain percent of them
PERCENT_DATA_USED = .003
endIdx = PERCENT_DATA_USED * len(training_data_init)
endIdx = int(endIdx)
training_data = training_data_init[:endIdx]

endIdx = PERCENT_DATA_USED * len(training_labels_init)
endIdx = int(endIdx)
trainingLabels = training_labels_init[:endIdx]



In [ ]:
# Tokenize the tweets (convert sentence to sequence of words)
print("2 -- Tokenizing the tweets: converting sentences to sequence of words")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(training_data)

sequences = tokenizer.texts_to_sequences(training_data)
word_index = tokenizer.word_index

2 -- Tokenizing the tweets: converting sentences to sequence of words


In [ ]:
# Pad sequences to ensure samples are the same size
print("3 -- Padding sequences to ensure samples are the same size")
training_data = pad_sequences(sequences)

3 -- Padding sequences to ensure samples are the same size


In [ ]:
print("4 -- Loading pre-trained word embeddings. This may take a few minutes.")

embeddings_index = {}
f = open(EMBEDDINGS_DIR,'rb')
for line in f:
    values = line.split()
    word = values[0].decode('UTF-8')
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

4 -- Loading pre-trained word embeddings. This may take a few minutes.


In [ ]:
print("5 -- Finding word embeddings for words in our tweets.")
# prepare word embedding matrix
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


5 -- Finding word embeddings for words in our tweets.


In [ ]:
print("Training Data Size: ", training_data.shape)
print("Number of Tweets: ", training_data.shape[0])
print("Max Tweet Length: ", training_data.shape[1])
print()
# print("Labels Size: ", labels.shape)
print(training_data.shape[1])
print(training_data[5])

Training Data Size:  (4800, 34)
Number of Tweets:  4800
Max Tweet Length:  34

34
[   0    0    0    0    0    0    0    0    0 3472 1646 1154   10 1344
    8   18 3473 2217    2 3474  103   65  115 2218 1647   11    3  227
  579    9   67  254  639   13]


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras import metrics
from keras.models import Model
import pickle
import tensorflow as tf


In [ ]:
trainingLabels2 = []
for i in trainingLabels:
  if i == '0':
    trainingLabels2.append(0)
  else:
    trainingLabels2.append(1)

print(trainingLabels2[4])

1


In [ ]:
training_data = np.array(training_data)
trainingLabels2 = np.array(trainingLabels2)
# print(type(trainingLabels[1]))
# print(trainingLabels[1])

In [ ]:
model = Sequential()


model.add(Embedding(len(word_index) + 1,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=training_data.shape[1],
                    trainable=False))

model.add(LSTM(EMBEDDING_DIM,recurrent_dropout=0,activation='relu',return_sequences=True))
model.add(LSTM(64,dropout=.1,recurrent_dropout=.1,activation='relu'))

model.add(Dense(1, activation="sigmoid"))

print(model.summary())

LOSS = 'binary_crossentropy'
OPTIMIZER = 'adam'

model.compile(loss = LOSS, optimizer = OPTIMIZER, metrics = [metrics.binary_accuracy])

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 34, 50)            571400    
                                                                 
 lstm_44 (LSTM)              (None, 34, 50)            20200     
                                                                 
 lstm_45 (LSTM)              (None, 64)                29440     
                                                                 
 dense_22 (Dense)            (None, 1)                 65        
                                                                 
Total params: 621,105
Trainable params: 49,705
Non-trainable params: 571,400
_________________________________________________________________
None


In [ ]:
#assisted from https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/
from sklearn.model_selection import train_test_split
seed = 7
np.random.seed(seed)

X_train, X_test, label_train, label_test = train_test_split(training_data, trainingLabels2, test_size=0.33, random_state=seed)


In [ ]:
TEST_SIZE = 0.5

#####################################
# TODO: pick number of epochs and batch size

EPOCHS = 10
BATCH_SIZE = 64

#####################################

model.fit(X_train, label_train, epochs = EPOCHS, batch_size = BATCH_SIZE, validation_split =TEST_SIZE)


Epoch 1/10
26/26 [==============================] - 20s 555ms/step - loss: 0.6901 - binary_accuracy: 0.5354 - val_loss: 0.6834 - val_binary_accuracy: 0.5914
Epoch 2/10
26/26 [==============================] - 14s 522ms/step - loss: 0.6745 - binary_accuracy: 0.6007 - val_loss: 0.6599 - val_binary_accuracy: 0.6175
Epoch 3/10
26/26 [==============================] - 13s 521ms/step - loss: 0.6654 - binary_accuracy: 0.6076 - val_loss: 0.6665 - val_binary_accuracy: 0.5852
Epoch 4/10
26/26 [==============================] - 13s 512ms/step - loss: 0.6404 - binary_accuracy: 0.6598 - val_loss: 0.6479 - val_binary_accuracy: 0.6437
Epoch 5/10
26/26 [==============================] - 14s 542ms/step - loss: 0.6264 - binary_accuracy: 0.6766 - val_loss: 0.6231 - val_binary_accuracy: 0.6629
Epoch 6/10
26/26 [==============================] - 13s 516ms/step - loss: 0.5976 - binary_accuracy: 0.7009 - val_loss: 0.6176 - val_binary_accuracy: 0.6704
Epoch 7/10
26/26 [==============================] - 14s 52

In [ ]:
results = model.evaluate(X_test,label_test,batch_size=128)


print("Final test cost/loss: ", results[0])
print("Final test accuracy: ",  results[1])


13/13 [==============================] - 0s 37ms/step - loss: 0.6074 - binary_accuracy: 0.6881
Final test cost/loss:  0.607373833656311
Final test accuracy:  0.6881313323974609
